# 1.1 Baseline Model

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

# Load your dataframe
df = pd.read_csv("/home/shuja3/code/gritchou/text2play/data/raw/paintings_dataset.csv")  # Replace "your_dataframe.csv" with your dataframe's filename
nan = df['Description'].notna()
df = df[nan]
df['Description'][1]

'Van Gogh\'s night sky is a field of roiling energy. Below the exploding stars, the village is a place of quiet order. Connecting earth and sky is the flamelike cypress, a tree traditionally associated with graveyards and mourning. But death was not ominous for van Gogh. "Looking at the stars always makes me dream," he said, "Why, I ask myself, shouldn\'t the shining dots of the sky be as accessible as the black dots on the map of France? Just as we take the train to get to Tarascon or Rouen, we take death to reach a star."\r\nThe artist wrote of his experience to his brother Theo: "This morning I saw the country from my window a long time before sunrise, with nothing but the morning star, which looked very big." This morning star, or Venus, may be the large white star just left of center in The Starry Night. The hamlet, on the other hand, is invented, and the church spire evokes van Gogh\'s native land, the Netherlands. The painting, like its daytime companion, The Olive Trees, is roo

In [5]:

# Preprocess the descriptions
# For simplicity, let's assume you have a column named 'description' in your dataframe
df['Description'] = df['Description'].str.lower()  # Convert to lowercase
df['Description'] = df['Description'].apply(lambda x: re.sub(r'[^\w\s]', '', x))  # Remove punctuation

# User input
user_input = "Existential Crisis"

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Description'])

# Convert user input to TF-IDF vector
user_input_vector = tfidf_vectorizer.transform([user_input])

# Compute cosine similarity between user input and descriptions
cos_similarities = cosine_similarity(user_input_vector, tfidf_matrix)

# Find the index of the most similar Description
most_similar_index = cos_similarities.argmax()

# Get the most similar Description
most_similar_description = df.loc[most_similar_index, 'Description']
print("Most Similar Description:", most_similar_description)


Most Similar Description: an interesting story that indicates kandinskys intellectual examination in the creation of his works revolves around this composition he became exhausted during the months of studying he went through in preparation for this painting and decided to go for a walk his assistant at the time gabrielle munter who was tidying up the studio in the artists absence inadvertently turned the canvas on its side upon kandinskys return he saw the canvas fell to his knees and began weeping at the beauty of the painting his newly found perspective on the piece would change his artistic vision and direction for the rest of his life 


## BERT Model


In [9]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np
import re

# Load your dataset
df = pd.read_csv('/home/shuja3/code/gritchou/text2play/data/raw/paintings_dataset.csv')# Update the path and file name as necessary
nan = df['Description'].notna()
df = df[nan]

In [10]:
# Cleaning
df['Description'] = df['Description'].str.lower()  # Convert to lowercase
print(df['Description'][1])

van gogh's night sky is a field of roiling energy. below the exploding stars, the village is a place of quiet order. connecting earth and sky is the flamelike cypress, a tree traditionally associated with graveyards and mourning. but death was not ominous for van gogh. "looking at the stars always makes me dream," he said, "why, i ask myself, shouldn't the shining dots of the sky be as accessible as the black dots on the map of france? just as we take the train to get to tarascon or rouen, we take death to reach a star."
the artist wrote of his experience to his brother theo: "this morning i saw the country from my window a long time before sunrise, with nothing but the morning star, which looked very big." this morning star, or venus, may be the large white star just left of center in the starry night. the hamlet, on the other hand, is invented, and the church spire evokes van gogh's native land, the netherlands. the painting, like its daytime companion, the olive trees, is rooted in 

In [12]:
df['Description'] = df['Description'].apply(lambda x: re.sub(r'[^\w\s]', '', x))  # Remove punctuation
print(df['Description'][1])


descriptions = df['Description'].tolist()  # Update 'description' if using a different column name

# Load the BERT model from sentence-transformers
model = SentenceTransformer('all-MiniLM-L6-v2')  # A smaller, faster model than BERT for testing
# model = SentenceTransformer('google-bert/bert-base-uncased')

van goghs night sky is a field of roiling energy below the exploding stars the village is a place of quiet order connecting earth and sky is the flamelike cypress a tree traditionally associated with graveyards and mourning but death was not ominous for van gogh looking at the stars always makes me dream he said why i ask myself shouldnt the shining dots of the sky be as accessible as the black dots on the map of france just as we take the train to get to tarascon or rouen we take death to reach a star
the artist wrote of his experience to his brother theo this morning i saw the country from my window a long time before sunrise with nothing but the morning star which looked very big this morning star or venus may be the large white star just left of center in the starry night the hamlet on the other hand is invented and the church spire evokes van goghs native land the netherlands the painting like its daytime companion the olive trees is rooted in imagination and memory leaving behind

/home/shuja3/.pyenv/versions/3.10.6/envs/text2play/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/shuja3/.pyenv/versions/3.10.6/envs/text2play/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [33]:
# Generate embeddings for the descriptions
description_embeddings = model.encode(descriptions, convert_to_tensor=True)

# Input text
input_text = "Futuristic city"
input_embedding = model.encode(input_text, convert_to_tensor=True)

# Calculate cosine similarities
cosine_scores = util.pytorch_cos_sim(input_embedding, description_embeddings)
type(cosine_scores)

torch.Tensor

In [34]:
# Convert the PyTorch tensor to a NumPy array
cosine_scores_np = cosine_scores.cpu().detach().numpy()

# Find the index of the highest scoring painting description
highest_score_index = np.argmax(cosine_scores_np)

# Output the most similar description and its score
most_similar_url = df.iloc[highest_score_index]
highest_score = cosine_scores[0, highest_score_index].item()

print("Relevant URL:", most_similar_url['Image URL'])
print("Similarity score:", highest_score)

Relevant URL: https://uploads2.wikiart.org/00129/images/edward-hopper/nighthawks.jpg!Large.jpg
Similarity score: 0.25323590636253357
